In [3]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import Timedelta

In [4]:
API_KEY = "PK1KAWB732Z95VFMPIEB"
API_SECRET = "lRxdiVh4jOoyZQvgUdqplHNuQZWI9apIPIXPaoaM"
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER": True
}

## ML Components for news (Sentimental Analysis)

### Importing transformer and testing the sentimental Analysis

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from typing import Tuple

In [6]:
device = "cudo:0" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)
labels = ["positive", "negative", "neutral"]

In [7]:
def estimate_sentiment(news):
    if news:
        tokens = tokenizer(news, return_tensors="pt", padding=True).to(device)

        result = model(tokens["input_ids"], attention_mask = tokens["attention_mask"])["logits"]

        result = torch.nn.functional.softmax(torch.sum(result, 0), dim = -1)
        probability = result[torch.argmax(result)]
        sentiment = labels[torch.argmax(result)]
        return probability, sentiment
    
    else:
        return 0, labels[-1]
    

In [8]:
if __name__ == "__main__":
    tensor, sentiment = estimate_sentiment(['markets responded negatively to the news!','traders were displeased!'])
    print(tensor, sentiment)
    print(torch.cuda.is_available())

tensor(0.9979, grad_fn=<SelectBackward0>) negative
False


## Backtesting Stocks using ML 

In [9]:


class MLTrader(Strategy):
    def initialize(self, symbol:str="GOOGL", cash_at_risk:float = .5):
        self.cash_at_risk = cash_at_risk
        self.symbol = symbol
        self.sleeptime = "24H"
        self.last_trade = None
        self.api = REST(base_url=BASE_URL, 
                        key_id=API_KEY, 
                        secret_key=API_SECRET)
    
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)

        return cash, last_price, quantity
    
    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days = 3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')
    
    def get_sentiment(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start = three_days_prior, 
                                 end = today )
        
        news = [ev.__dict__["_raw"]["headline"] for ev in news]

        probability, sentiment  = estimate_sentiment(news)
            
        return probability, sentiment


    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        if cash > last_price:
            if self.last_trade == None:
                probability, sentiment = self.get_sentiment()
                print(probability, sentiment)
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="bracket",
                    take_profit_price=last_price*1.20,
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order)
                self.last_trade = "buy"
        




In [10]:
brokers = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat', broker=brokers, parameters={"symbol": "GOOGL", 
                                                                "cash_at_risk":.5})

start_date = datetime(2023,12,15)
end_date = datetime(2023,12,31)

strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol": "GOOGL", 
                "cash_at_risk":.5}
)

INFO:backtest_stats:Starting backtest...


Starting backtest for MLTrader...
Progress ||   2.47%  [Elapsed: 0:00:01 ETA: 0:00:47] tensor(1., grad_fn=<SelectBackward0>) neutral
Progress || 100.00%  [Elapsed: 0:00:03 ETA: 0:00:00] Portfolio Val: 103,043.80
Creating trades plot...

Creating indicators plot...

Creating tearsheet...


{'cagr': 1.0753092333495213,
 'volatility': 0.09183644996007587,
 'sharpe': 11.138379544102913,
 'max_drawdown': {'drawdown': 0.007176093998772082,
  'date': Timestamp('2023-12-29 09:30:00-0500', tz='America/New_York')},
 'romad': 149.84603511792346,
 'total_return': 0.03043803710937465}

In [1]:
# pip install alpaca-trade-api==3.1.1
# pip install timedelta
# pip install install torch torchvision torchaudio transformers